In [1]:
import requests
%matplotlib inline
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json
import urllib
import numpy as np
import string
import pyodbc

In [2]:
cnxn = pyodbc.connect(DSN="ACS",ansi=True)
df = pd.io.sql.read_sql_query(""" 
    SELECT CMPN_CD, CHANNEL, SUM(VAL_OF_DLVRY) AS VOD, SUM(CASH_SPEND) AS CASH
    FROM BACC_PRD_IDM_ACS.ACS_ADVR2.V_FCT_PAID_MEDIA
    WHERE CMPN_CD LIKE '000%'
    AND DT BETWEEN (CURRENT_DATE -90) AND CURRENT_DATE
    GROUP BY CMPN_CD, CHANNEL
    ORDER BY CMPN_CD; """, con=cnxn) 
cnxn.close()
df.head(5)

,CMPN_CD,CHANNEL,VOD,CASH
0,000000MI,Paid Search,104929.760036,101135.840004
1,000000MI,Display,21889.830781,21889.830781
2,000000MI,Paid Social,56641.024160,56641.024160
3,000000MI,Content Syndication,35314.120000,25921.000000
4,000000MM,Paid Search,39988.340004,39251.580002


In [2]:
from ibmdbpy import IdaDataBase, IdaDataFrame
cnxn = IdaDataBase(dsn="V2PPRD")

df2 = cnxn.ida_query("""SELECT MAX(DT) FROM V2PSTG.FCT_PAID_MEDIA""")

cnxn.close()
df2.head()

ProgrammingError: ('42501', '[42501] [IBM][CLI Driver][DB2/LINUXX8664] SQL0551N  The statement failed because the authorization ID does not have the required authorization or privilege to perform the operation.  Authorization ID: "NAZU".  Operation: "SELECT". Object: "V2PSTG.FCT_PAID_MEDIA".  SQLSTATE=42501\r\n')

In [6]:
df2.to_csv('C:\\Users\IBM_ADMIN\Documents\Alerts\ALERT 1 - Zero Spend\VOD\db2_20180827.csv', header=True, index=False, index_label=None)

In [8]:
df3 = pd.merge(df, df2, left_on=['CMPN_CD','CHANNEL'], right_on=['proj_cd','marktng_channl_type'], how='right')
df3.sort_values('proj_cd')
df3.head(5)

,CMPN_CD,CHANNEL,VOD,CASH,load_dt,start_window,end_window,proj_id,proj_cd,proj_name,...,marktng_channl_type,paid_media_imprsn_cnt,paid_media_click_cnt,paid_media_cost_usd,paid_visit_cnt,paid_engaged_visit_cnt,tot_reg_cnt,respns_cnt,high_qlty_resp_cnt,alert_id
0,000000MI,Paid Search,104929.760036,101135.840004,2018-08-25,2018-05-27,2018-08-25,49520,000000MI,000000MI_WW_Detect Threats with Analytics,...,Paid Search,152457,242,0.0,3870,1230.0,263,211,161,PM_1
1,000000MI,Display,21889.830781,21889.830781,2018-08-25,2018-05-27,2018-08-25,49520,000000MI,000000MI_WW_Detect Threats with Analytics,...,Display,532322,1362,0.0,1396,15.0,2,2,1,PM_1
2,000000MI,Paid Social,56641.024160,56641.024160,2018-08-25,2018-05-27,2018-08-25,49520,000000MI,000000MI_WW_Detect Threats with Analytics,...,Paid Social,5740,0,0.0,11399,1030.0,87,79,23,PM_1
3,000000MI,Content Syndication,35314.120000,25921.000000,2018-08-25,2018-05-27,2018-08-25,49520,000000MI,000000MI_WW_Detect Threats with Analytics,...,Content Syndication,0,0,0.0,0,0.0,1272,1266,203,PM_1
4,000000MM,Paid Search,39988.340004,39251.580002,2018-08-25,2018-05-27,2018-08-25,49524,000000MM,000000MM_WW_Network protection_BigFix,...,Paid Search,40248,103,0.0,2229,709.0,26,20,14,PM_1


In [10]:
df3.to_csv('C:\\Users\IBM_ADMIN\Documents\Alerts\ALERT 1 - Zero Spend\VOD\\vod_raw_20180827.csv', header=True, index=False, index_label=None)

In [11]:
del df3['CMPN_CD']
del df3['CHANNEL']
del df3['CASH']
df4 = df3[df3.proj_cd.notnull()]
df4['VOD'] = df4.VOD.replace(np.NaN, 0)
df4['VOD'] = df4.VOD.replace("0", 0)
df5 = df4[df4.VOD == 0]
df5.head(5)

,VOD,load_dt,start_window,end_window,proj_id,proj_cd,proj_name,proj_state_cd,proj_start_dt,proj_end_dt,...,marktng_channl_type,paid_media_imprsn_cnt,paid_media_click_cnt,paid_media_cost_usd,paid_visit_cnt,paid_engaged_visit_cnt,tot_reg_cnt,respns_cnt,high_qlty_resp_cnt,alert_id
47,0.0,2018-08-25,2018-05-27,2018-08-25,49550,000000OH,000000OH B2B Cloud Services New Buyer,IN_PROGRESS,2016-04-01,2018-12-31,...,Display,3,0,0.0,0,0.0,0,0,0,PM_1
48,0.0,2018-08-25,2018-05-27,2018-08-25,49433,000000OI,000000OI MFT Consolidation New Buyer,IN_PROGRESS,2016-04-01,2018-12-31,...,Display,49,0,0.0,0,0.0,0,0,0,PM_1
50,0.0,2018-08-25,2018-05-27,2018-08-25,49552,000000ON,000000ON WW Security Operations and Response,IN_PROGRESS,2016-02-22,2018-12-31,...,Display,0,0,0.0,3,NaN,0,0,0,PM_1
51,0.0,2018-08-25,2018-05-27,2018-08-25,49435,000000OS,000000OS Watson Explorer,IN_PROGRESS,2016-02-24,2018-01-12,...,Display,151,0,0.0,0,0.0,0,0,0,PM_1
56,0.0,2018-08-25,2018-05-27,2018-08-25,49576,000000PY,000000PY WW IBM Connections Verse New,IN_PROGRESS,2016-05-12,2018-12-31,...,Paid Social,0,53,0.0,3,1.0,0,0,0,PM_1


In [14]:
df5.to_csv('C:\\Users\IBM_ADMIN\Documents\Alerts\ALERT 1 - Zero Spend\VOD\\vod_raw2_20180827.csv', header=True, index=False, index_label=None)

In [15]:
df5.columns

Index(['VOD', 'load_dt', 'start_window', 'end_window', 'proj_id', 'proj_cd',
       'proj_name', 'proj_state_cd', 'proj_start_dt', 'proj_end_dt',
       'top_proj_cd', 'top_proj_dscr', 'prog_cd', 'prog_name', 'prog_start_dt',
       'prog_end_dt', 'plan_name', 'plan_cd', 'plan_start_dt', 'plan_end_dt',
       'origntng_mktg_team_cd', 'origntng_mktg_team_name',
       'bus_unit_gbtl10_cd', 'marktng_channl_type', 'paid_media_imprsn_cnt',
       'paid_media_click_cnt', 'paid_media_cost_usd', 'paid_visit_cnt',
       'paid_engaged_visit_cnt', 'tot_reg_cnt', 'respns_cnt',
       'high_qlty_resp_cnt', 'alert_id'],
      dtype='object')

In [16]:
import pandasql as ps

df6 = """
    SELECT 
        alert_id
        ,start_window
        ,end_window
        ,load_dt
        ,proj_id
        ,proj_cd
        ,proj_name
        ,proj_state_cd
        ,proj_start_dt
        ,proj_end_dt
        ,top_proj_cd
        ,top_proj_dscr
        ,prog_cd
        ,prog_name
        ,prog_start_dt
        ,prog_end_dt
        ,plan_cd
        ,plan_start_dt
        ,plan_end_dt
        ,origntng_mktg_team_cd
        ,origntng_mktg_team_name
        ,bus_unit_gbtl10_cd
        ,marktng_channl_type
        ,SUM(paid_media_imprsn_cnt) AS IMPRESSIONS
        ,SUM(paid_media_click_cnt) AS CLICKS
        ,SUM(VOD) AS spend
        ,SUM(paid_visit_cnt) AS visits
        ,SUM(paid_engaged_visit_cnt) AS ENGAGED_VISITS
        ,SUM(tot_reg_cnt) AS REGISTRATIONS
        ,SUM(respns_cnt) AS RESPONSES
        ,SUM(high_qlty_resp_cnt) AS HQ_RESPONSES
        
        FROM df5 

        GROUP BY
        alert_id
        ,start_window
        ,end_window
        ,load_dt
        ,proj_id
        ,proj_cd
        ,proj_name
        ,proj_state_cd
        ,proj_start_dt
        ,proj_end_dt
        ,top_proj_cd
        ,top_proj_dscr
        ,prog_cd
        ,prog_name
        ,prog_start_dt
        ,prog_end_dt
        ,plan_name
        ,plan_cd
        ,plan_start_dt
        ,plan_end_dt
        ,origntng_mktg_team_cd
        ,origntng_mktg_team_name
        ,bus_unit_gbtl10_cd
        ,marktng_channl_type; """

df7 = (ps.sqldf(df6, locals()))
df7.head(5)

,alert_id,start_window,end_window,load_dt,proj_id,proj_cd,proj_name,proj_state_cd,proj_start_dt,proj_end_dt,...,bus_unit_gbtl10_cd,marktng_channl_type,IMPRESSIONS,CLICKS,spend,visits,ENGAGED_VISITS,REGISTRATIONS,RESPONSES,HQ_RESPONSES
0,PM_1,2018-05-27,2018-08-25,2018-08-25,49433,000000OI,000000OI MFT Consolidation New Buyer,IN_PROGRESS,2016-04-01,2018-12-31,...,None,Display,49,0,0.0,0,0.0,0,0,0
1,PM_1,2018-05-27,2018-08-25,2018-08-25,49435,000000OS,000000OS Watson Explorer,IN_PROGRESS,2016-02-24,2018-01-12,...,None,Display,151,0,0.0,0,0.0,0,0,0
2,PM_1,2018-05-27,2018-08-25,2018-08-25,49435,000000OS,000000OS Watson Explorer,IN_PROGRESS,2016-02-24,2018-01-12,...,None,Paid Social,0,0,0.0,84,NaN,0,0,0
3,PM_1,2018-05-27,2018-08-25,2018-08-25,49454,000000QP,000000QP_WW_Experience Design_iX,IN_PROGRESS,2016-03-31,2017-12-31,...,GBS,Paid Search,0,0,0.0,2,NaN,0,0,0
4,PM_1,2018-05-27,2018-08-25,2018-08-25,49455,000000QR,000000QR_WW_Datacenter IT support_TSS MVS,IN_PROGRESS,2016-04-05,2017-12-31,...,GTS,Paid Social,0,0,0.0,33,NaN,0,0,0


In [18]:
df7.to_csv('C:\\Users\IBM_ADMIN\Documents\Alerts\ALERT 1 - Zero Spend\VOD\\vod_20180827.csv', header=True, index=False, index_label=None)